We first get necessary external data and code

In [1]:
!git clone https://github.com/AllenInstitute/deepinterpolation.git
!mkdir -p ephys

Cloning into 'deepinterpolation'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (48/48), done.


Install deepinterpolation package

In [19]:
!pip install git+https://github.com/AllenInstitute/deepinterpolation.git

  Cloning https://github.com/AllenInstitute/deepinterpolation.git to /private/var/folders/hy/9t9mb1ln50z59qlgy6nc7y7r3s796j/T/pip-req-build-zthhkwzm
  Created wheel for deepinterpolation: filename=deepinterpolation-0.1.0-py3-none-any.whl size=21725 sha256=10259817285bfa10cf2d9ab267b92bca6aed56e616fb5a36de6f91afc5e7ac40
  Stored in directory: /private/var/folders/hy/9t9mb1ln50z59qlgy6nc7y7r3s796j/T/pip-ephem-wheel-cache-ullpkl4l/wheels/8e/c0/d8/0c84568edf2d461bf07b4e88eacb17b4ca3765e331f319c3b6
Successfully built deepinterpolation


In [1]:
import deepinterpolation as de
import os
from deepinterpolation.generator_collection import EphysGenerator
from deepinterpolation.trainor_collection import transfer_trainer
import datetime

This is used for record-keeping


In [2]:
now = datetime.datetime.now()
run_uid = now.strftime("%Y_%m_%d_%H_%M")

Initialize meta-parameters objects

In [3]:
training_param = {}
generator_param = {}
network_param = {}
generator_test_param = {}

An epoch is defined as the number of batches pulled from the dataset. Because our datasets are VERY large. Often, we cannot
go through the entirity of the data so we define an epoch slightly differently than is usual.

In [4]:
steps_per_epoch = 10

Those are parameters used for the Validation test generator. Here the test is done on the beginning of the data but
this can be a separate file

In [14]:
generator_test_param[
    "pre_post_frame"
] = 30  # Number of frame provided before and after the predicted frame
generator_test_param["train_path"] = os.path.join(
    "..",
    "sample_data",
    "ephys_tiny_continuous.dat2",
)
generator_test_param["batch_size"] = 100
generator_test_param["start_frame"] = 0
generator_test_param["end_frame"] = 1999
generator_test_param[
    "pre_post_omission"
] = 1  # Number of frame omitted before and after the predicted frame
generator_test_param["steps_per_epoch"] = -1  # No step necessary for testing as epochs are not relevant. -1 deactivate it.

Those are parameters used for the main data generator

In [15]:
generator_param["steps_per_epoch"] = steps_per_epoch
generator_param["pre_post_frame"] = 30
generator_param["train_path"] = os.path.join(
    "..",
    "sample_data",
    "ephys_tiny_continuous.dat2",
)
generator_param["batch_size"] = 100
generator_param["start_frame"] = 2000
generator_param["end_frame"] = 7099
generator_param["pre_post_omission"] = 1

Those are parameters used for the training process

In [19]:
training_param["run_uid"] = run_uid

# Path to model to transfer and fine-tune
training_param["model_path"] = "../sample_data/2020_02_29_15_28_unet_single_ephys_1024_mean_squared_error-1050.h5"
training_param["batch_size"] = generator_test_param["batch_size"]
training_param["steps_per_epoch"] = steps_per_epoch
training_param[
    "period_save"
] = 25  # network model is potentially saved during training between a regular nb epochs
training_param["nb_gpus"] = 0
training_param["apply_learning_decay"] = 0
training_param[
    "nb_times_through_data"
] = 1  # if you want to cycle through the entire data. Two many iterations will cause noise overfitting
training_param["learning_rate"] = 0.0001
training_param["pre_post_frame"] = generator_test_param["pre_post_frame"]
training_param["loss"] = "mean_absolute_error"
training_param[
    "nb_workers"
] = 1  # this is to enable multiple threads for data generator loading. Useful when this is slower than training

training_param["model_string"] = (
    "transfer"
    + "_"
    + training_param["loss"]
    + "_"
    + training_param["run_uid"]
)

Where do you store ongoing training progress

In [20]:
jobdir = os.path.join(
    training_param["model_string"] + "_" + run_uid,
)
training_param["output_dir"] = jobdir

try:
    os.mkdir(jobdir)
except:
    print("folder already exists")

folder already exists


Here we create all json files that are fed to the training. This is used for recording purposes as well as input to the training proces

Here we create all objects for training.

In [24]:
# We build the generators object. This will, among other things, calculate normalizing parameters.
train_generator = EphysGenerator(generator_param)
test_generator = EphysGenerator(generator_test_param)

# We build the training object.
training_class = transfer_trainer(train_generator, test_generator, training_param)

19/19 [==============================] - 4s 195ms/step - loss: 0.3053


Start training. This can take very long time.

In [22]:
training_class.run()

Epoch 1/5
10/10 [==============================] - 15s 1s/step - loss: 0.3487 - val_loss: 0.3107
Epoch 2/5
10/10 [==============================] - 15s 1s/step - loss: 0.3200 - val_loss: 0.3105
Epoch 3/5
10/10 [==============================] - 15s 1s/step - loss: 0.3177 - val_loss: 0.3105
Epoch 4/5
10/10 [==============================] - 15s 1s/step - loss: 0.3159 - val_loss: 0.3107
Epoch 5/5
10/10 [==============================] - 14s 1s/step - loss: 0.3149 - val_loss: 0.3100


Finalize and save output of the training.

In [23]:
training_class.finalize()

Saved model to disk


/Users/jerome.lecoq/opt/miniconda3/envs/deepinterpolation_new/lib/python3.10/site-packages/keras-2.15.0-py3.10.egg/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
